In [ ]:
%matplotlib widget
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage
import h5py
from IPython.display import Video
from IPython.display import HTML
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar

In [ ]:
import hyperspy.api as hs

## Fine tune ptyrex reconstruction parameters and resubmit the job Widget

In [ ]:
%matplotlib widget
import sys
sys.path.append('/dls/science/groups/e02/Frederick/epsic_tools_2025/epsic_tools/epsic_tools/mib2hdfConvert/MIB_convert_widget/scripts/')
from MIB_convert import *
os.chdir('/dls/e01/data/2025/cm40602-5/raw/Ptycho_Pt-C_session')
ptyrex_meta = convert_info_widget(ptyrex_single=True)

## Select a reconstruction to view and perform post processing

In [ ]:
sys.path.append('/dls/science/groups/e02/Frederick/0_Ptychography_tools_for_users/')
import Ptyrex_postprocess as post

In [ ]:
path = os.getcwd()
a = post.Recon_output_helper()

In [ ]:
#the output of the ptychography reconstuction some times needs a flag to be clear in order to read. The code below below clears this flag.
post.h5clear_flags_series([a.recon_path]) #Note that a.recon_path has been in between square brackets [] such that it is a list

# H5Web viewer

### to view to the object phase:
click the following: entery_1 -> process_1 -> output_1 -> object_phase

In [ ]:
#This cell is currently not working at the moment...
from jupyterlab_h5web import H5Web
H5Web(a.recon_path)

## Load data

In [ ]:
#load the reconstructed phase
with h5py.File(a.recon_path,'r') as f:
    object_phase = f['entry_1']['process_1']['output_1']['object_phase'][()]

print(f'shape before: {np.shape(object_phase)}')
#reducing the object down from 7d array to 3d array
object_phase = object_phase[0,:,0,0,0,:,:]
print(f'shape after:  {np.shape(object_phase)}')

In [ ]:

#rotate the each object slice
object_phase_rot = np.zeros(np.shape(object_phase))
for counter1 in np.arange(np.shape(object_phase)[0]):
    object_phase_rot[counter1,:,:] = skimage.transform.rotate(np.copy(object_phase[counter1,:,:]),85.5)



In [ ]:
#plot a particular slice of the 3d numpy array using matplotlib
index = 0
plt.figure()
plt.imshow(object_phase_rot[index,:,:],cmap='inferno')

In [ ]:
# the below function is just np.fft.fftshift(np.fft.fft2(np.fft.fftshift(array)))
object_phase_rot_fft = post.quick_fft(object_phase_rot)

In [ ]:
#create figure 
fig,axs = plt.subplots(1,2)
#plot object slice and its fft
axs[0].imshow(object_phase_rot[index,:,:],cmap='inferno')
axs[1].imshow(np.abs(object_phase_rot_fft[index,:,:]),norm='symlog',cmap='inferno')
#clear up the figure
axs[0].xaxis.set_visible(False)
axs[0].yaxis.set_visible(False)
axs[1].xaxis.set_visible(False)
axs[1].yaxis.set_visible(False)
#create figure labels
axs[0].text(5,90,'a)',color='w')
axs[1].text(5,90,'b)',color='w')
#create scale bars real space
pixel_size = post.QuickPixelSize(a.recon_path)
number_of_pixels = 2e-9/pixel_size
bar = AnchoredSizeBar(axs[0].transData, number_of_pixels, '2 nm', loc='lower center')
axs[0].add_artist(bar)
#create scale bar Fourier space
F = 1/pixel_size #F is the width of the array in Fouirier space
df = F/np.shape(object_phase_rot)[-1] #df Fourier space pixel_size is determined by the number of sampling points in real space 
                                      #so if you crop the real space image the smapling gets worse in fourier space
num_pix_fft = (1/1e-10)/df
bar_fft = AnchoredSizeBar(axs[0].transData, num_pix_fft, r'1 $\.A^{-1}$', loc='lower center')
axs[1].add_artist(bar_fft)


In [ ]:
# lets try some windowing function to clean up the fft
from skimage.filters import window
object_phase_rot_crop = object_phase_rot[:,450:1150,450:1150]
hann_filter = window('hann', object_phase_rot_crop.shape[1:])
warray = object_phase_rot_crop * hann_filter
warray_fft = post.quick_fft(warray)

#this function makes a mosiac of plots of different sizes
fig,axs2 = plt.subplot_mosaic(
    '''
    aabb
    cccc
    cccc
    '''
)

axs2['a'].imshow(warray[index,:,:],cmap='inferno')
axs2['b'].imshow(hann_filter,cmap='inferno')
axs2['c'].imshow(np.abs(warray_fft[index,:,:]),norm='symlog',cmap='inferno')

Note that the peaks in the FFT are now smaller and do not have the cross shape they had before. this is due to the hann window function reducing the discontinuities within the reconstruction array 

some explaination of windowing functions:

https://scikit-image.org/docs/0.25.x/auto_examples/filters/plot_window.html

https://download.ni.com/evaluation/pxi/Understanding%20FFTs%20and%20Windowing.pdf